In [ ]:
import symforce

symforce.set_symbolic_api("sympy")
symforce.set_log_level("warning")

import symforce.symbolic as sf
from symforce.ops import StorageOps
from symforce.ops import LieGroupOps

epsilon = 1e-9

In [ ]:
def tangent_D_storage(a):
    """
    Computes the jacobian of the storage space of an element with respect to the tangent space around
    that element.
    """
    # Perturb a in the storage space
    storage_dim = LieGroupOps.storage_dim(a)
    xi = sf.M(storage_dim, 1).symbolic("xi")
    storage_perturbed = sf.M(LieGroupOps.to_storage(a)) + xi
    a_perturbed = LieGroupOps.from_storage(a, storage_perturbed)
    a_perturbed_tangent = sf.M(LieGroupOps.local_coordinates(a, a_perturbed))

    # Compute jacobian of storage wrt perturbation
    tangent_D_storage = a_perturbed_tangent.jacobian(xi)

    # Evaluate at perturbation == zero
    tangent_D_storage = tangent_D_storage.subs(xi, xi.zero())

    return tangent_D_storage

In [ ]:
def tangent_D_storage_approx(a, epsilon):
    """
    Computes the jacobian of the storage space of an element with respect to the tangent space around
    that element.

    This is an approximation - note that the exact jacobian can often be recovered with a call to
    nsimplify with the appropriate tolerance (though this requires the use of sympy rather than symengine)
    """
    # Perturb a in the storage space
    storage_dim = LieGroupOps.storage_dim(a)
    xi = sf.M(storage_dim, 1).symbolic("xi")
    storage_perturbed = sf.M(LieGroupOps.to_storage(a)) + xi
    a_perturbed = LieGroupOps.from_storage(a, storage_perturbed)
    a_perturbed_tangent = sf.M(LieGroupOps.local_coordinates(a, a_perturbed))

    # Compute jacobian of storage wrt perturbation
    tangent_D_storage = a_perturbed_tangent.jacobian(xi)

    # Rather than computing the limit, we substitude a small value for xi to approximate the limit
    # NOTE: This is much faster than taking the limit in sympy, but returns an approximation of the true
    # jacobian.
    assert epsilon != 0
    tangent_D_storage = tangent_D_storage.subs(xi, epsilon * xi.one())

    return tangent_D_storage

In [ ]:
rot2 = sf.Rot2.symbolic("A")
display(tangent_D_storage_approx(rot2, epsilon))
display(tangent_D_storage(rot2))
display(tangent_D_storage(rot2).subs(rot2.z.squared_norm(), 1).subs(rot2.z.squared_norm(), 1))

In [ ]:
rot3 = sf.Rot3().symbolic("A")
display(tangent_D_storage(rot3))
(
    tangent_D_storage(rot3)
    .subs(sf.Min(1, rot3.q.squared_norm()), rot3.q.squared_norm())
    .subs(sf.Max(0, 1 - rot3.q.squared_norm() ** 2), 1 - rot3.q.squared_norm() ** 2)
    .subs(rot3.q.squared_norm(), sf.Symbol("v"))
    .limit(sf.Symbol("v"), 1)
)

In [ ]:
pose2 = sf.Pose2.symbolic("A")
pose2_tangent_D_storage = pose2.storage_D_tangent().mat.pinv()
display(pose2_tangent_D_storage)
pose2_tangent_D_storage.subs(pose2.R.z.squared_norm(), 1)

In [ ]:
pose3 = sf.Pose3.symbolic("A")
pose3_tangent_D_storage = pose3.storage_D_tangent().mat.pinv()

# This takes a while
simplified = sf.simplify(pose3_tangent_D_storage)

In [ ]:
more_simplified = simplified.subs(pose3.R.q.squared_norm(), 1)
more_simplified

In [ ]:
# The bottom right block is equal to R^{-1}, which is just R^T
sf.simplify(pose3.R.to_rotation_matrix().matrix_inverse().mat) - more_simplified[3:, 4:]

For Pose3, if we don't want to wait for sympy, we can also get the result ourselves, because we know that Pose3's storage_D_tangent has the form:
$$
{}_S{D}_T^P = \begin{bmatrix}
{}_S{D}_T^R & 0 \\
0 & R
\end{bmatrix}
$$
where ${}_S{D}_T^R$ is Rot3's storage_D_tangent, and $R$ is the rotation matrix for Rot3.  Because ${}_S{D}_T^P$ has linearly independent columns, its pseudoinverse can be computed as $A^+ = (A^T A)^{-1} A^T$ (see https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse).  Writing this out,

$$
\begin{align}
{}_T{D}_S^P &= \left( {}_S{D}_T^P \right)^+ \\
&= \left(
\begin{bmatrix}
({}_S{D}_T^R)^T & 0 \\
0 & R^T
\end{bmatrix}
\begin{bmatrix}
{}_S{D}_T^R & 0 \\
0 & R
\end{bmatrix}
\right)^{-1}
\begin{bmatrix}
({}_S{D}_T^R)^T & 0 \\
0 & R^T
\end{bmatrix} \\
&= \begin{bmatrix}
({}_S{D}_T^R)^T {}_S{D}_T^R & 0 \\
0 & R^T R
\end{bmatrix}^{-1}
\begin{bmatrix}
({}_S{D}_T^R)^T & 0 \\
0 & R^T
\end{bmatrix} \\
&= \begin{bmatrix}
\frac{1}{4}({}_S{D}_T^R)^+ {}_S{D}_T^R & 0 \\
0 & \mathbb{1}
\end{bmatrix}^{-1}
\begin{bmatrix}
\frac{1}{4}({}_S{D}_T^R)^+ & 0 \\
0 & R^T
\end{bmatrix} \\
&= \begin{bmatrix}
\frac{1}{4} \mathbb{1} & 0 \\
0 & \mathbb{1}
\end{bmatrix}^{-1}
\begin{bmatrix}
\frac{1}{4} {}_T{D}_S^R & 0 \\
0 & R^T
\end{bmatrix} \\
&= \begin{bmatrix}
4 \mathbb{1} & 0 \\
0 & \mathbb{1}
\end{bmatrix}
\begin{bmatrix}
\frac{1}{4} {}_T{D}_S^R & 0 \\
0 & R^T
\end{bmatrix} \\
&= \begin{bmatrix}
{}_T{D}_S^R & 0 \\
0 & R^T
\end{bmatrix} \\
\end{align}
$$